# **(Data Visualization)**

In [ ]:
# Set Input Directories
my_data_dir = 'inputs/mildew_dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

In [ ]:
import os

# 作業ディレクトリの設定（Google Colabではルートディレクトリから開始することが多い）
work_dir = "/content"

# 出力フォルダのパス
output_dir = os.path.join(work_dir, 'outputs')
version = "v1.0"  # バージョン管理のためのフォルダ名

# 出力ディレクトリの作成
os.makedirs(output_dir, exist_ok=True)

file_path = os.path.join(output_dir, version)

# 既にフォルダが存在するかチェック
if 'outputs' in os.listdir(work_dir) and version in os.listdir(output_dir):
    print('Old version is already available. Create a new version.')
else:
    print(f'Creating new version directory: {file_path}')
    os.makedirs(file_path, exist_ok=True)

In [ ]:
import os

# 訓練データのディレクトリを設定
train_path = "/content/inputs/mildew_dataset/cherry-leaves/train"

# ディレクトリが存在するか確認
if os.path.exists(train_path):
    labels = os.listdir(train_path)
    print('✅ Label for the images are:', labels)
else:
    print(f"⚠ エラー: '{train_path}' が見つかりません。ディレクトリを確認してください。")

In [ ]:
# Image Shape

import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imageio import imread  # 画像読み込み用

# 訓練データのパス
train_path = "/content/inputs/mildew_dataset/cherry-leaves/train"

# 画像の平均サイズを計算
dim1, dim2 = [], []
if os.path.exists(train_path):
    labels = os.listdir(train_path)

    for label in labels:
        label_dir = os.path.join(train_path, label)
        for image_filename in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_filename)
            try:
                img = imread(image_path)
                d1, d2, _ = img.shape  # 高さ・幅・色チャネル数
                dim1.append(d1)  # 高さ
                dim2.append(d2)  # 幅
            except Exception as e:
                print(f"⚠ 画像読み込みエラー: {image_path}, {e}")

    # Seabornのスタイル設定
    sns.set_style("whitegrid")
    fig, axes = plt.subplots()

    # 画像のサイズを散布図としてプロット
    sns.scatterplot(x=dim2, y=dim1, alpha=0.2)
    axes.set_xlabel("Width (pixels)")
    axes.set_ylabel("Height (pixels)")

    # 平均サイズを計算
    dim1_mean = int(np.mean(dim1))
    dim2_mean = int(np.mean(dim2))

    # 平均値の線を描画
    axes.axvline(x=dim2_mean, color='r', linestyle='--')
    axes.axhline(y=dim1_mean, color='r', linestyle='--')

    plt.show()

    # 平均サイズを出力
    print(f"📏 平均 Width: {dim2_mean}px \n📏 平均 Height: {dim1_mean}px")

else:
    print(f"⚠ エラー: '{train_path}' が見つかりません。データセットを確認してください。")

In [ ]:
image_shape = (128, 128, 3)
image_shape

In [ ]:
import os
import joblib

file_path = "/content/outputs/v1.0"

# ✅ もし `image_shape.pkl` が存在する場合は削除
if os.path.exists(f"{file_path}/image_shape.pkl"):
    os.remove(f"{file_path}/image_shape.pkl")
    print("🗑️ `image_shape.pkl` を削除しました。")

# ✅ 正しい `image_shape` を (128, 128) で保存
image_shape = (128, 128)  # 修正: タプル形式で保存
joblib.dump(value=image_shape, filename=f"{file_path}/image_shape.pkl")

print(f"✅ `image_shape.pkl` を正しい形式で保存しました: {image_shape}")


In [ ]:
# ✅ `image_shape.pkl` をロード
image_shape = joblib.load(f"{file_path}/image_shape.pkl")

print("ロードした image_shape:", image_shape)
print("image_shapeの型:", type(image_shape))


In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# データセットのディレクトリ
my_data_dir = "/content/inputs/mildew_dataset/cherry-leaves"

# 出力フォルダのパス（Google Colab用）
file_path = "/content/outputs/v1.0"
os.makedirs(file_path, exist_ok=True)  # フォルダがない場合は作成

# クラスラベルの取得（trainセットから取得）
labels = os.listdir(os.path.join(my_data_dir, "train"))

# 2. クラス分布 (Training, Validation, Test)
data = {"Set": [], "Label": [], "Frequency": []}  # 修正

folders = ["train", "validation", "test"]

for folder in folders:
    for label in labels:
        label_path = os.path.join(my_data_dir, folder, label)
        frequency = len(os.listdir(label_path))
        data["Set"].append(folder)
        data["Label"].append(label)
        data["Frequency"].append(frequency)
        print(f"✅ {folder} - {label}: {frequency} images")

# Pandasデータフレームに変換
df_freq = pd.DataFrame(data)

# クラス分布をプロット
plt.figure(figsize=(8, 5))
sns.barplot(data=df_freq, x="Set", y="Frequency", hue="Label")
plt.title("Class Distribution Across Datasets (for Balance Check)")  # 目的を追加
plt.savefig(f"{file_path}/class_distribution.png")  # グラフを保存
plt.show()

### Average and Variability of Images Per Label

In [ ]:
from tensorflow.keras.preprocessing import image


def load_image_as_array(my_data_dir, new_size=(50, 50), n_images_per_label=20):
    """
    Loads images from a directory, resizes them, and returns them as arrays with labels.

    Parameters:
    - my_data_dir (str): Path to the dataset directory with subfolders for each label.
    - new_size (tuple): Target size for resizing images (default: (50, 50)).
    - n_images_per_label (int): Max number of images to load per label (default: 20).

    Returns:
    - X (numpy.ndarray): Array of resized image data.
    - y (numpy.ndarray): Array of corresponding labels.
    """

    X, y = np.array([], dtype="int"), np.array([], dtype="object")
    labels = os.listdir(my_data_dir)

    for label in labels:
        counter = 0
        for image_filename in os.listdir(my_data_dir + "/" + label):
            if counter < n_images_per_label:

                img = image.load_img(
                    my_data_dir + "/" + label + "/" + image_filename,
                    target_size=new_size,
                )
                if image.img_to_array(img).max() > 1:
                    img_resized = image.img_to_array(img) / 255
                else:
                    img_resized = image.img_to_array(img)

                X = np.append(X, img_resized).reshape(
                    -1, new_size[0], new_size[1], img_resized.shape[2]
                )
                y = np.append(y, label)
                counter += 1

    return X, y

In [ ]:
X, y = load_image_as_array(
    my_data_dir=train_path,
    new_size=image_shape,  # ✅ (128, 128) のタプルを渡す
    n_images_per_label=30,
)

print("Xの形状:", X.shape)
print("yの形状:", y.shape)

In [ ]:
print("image_shape:", image_shape)
print("image_shapeの型:", type(image_shape))

In [ ]:
def plot_mean_variability_per_labels(X, y, figsize=(12, 5), save_image=False):
    """
    The pseudo-code for the function is:
    * Iterate through all unique labels in the dataset.
    * Filter the dataset to include only images corresponding to the current label.
    * Calculate the mean and standard deviation for the filtered subset.
    * Create a figure with two subplots:
        - One displaying the mean image for the label.
        - The other showing the variability (standard deviation) image.
    * Optionally save the generated plots to the specified directory.
    """

    for label_to_display in np.unique(y):
        sns.set_style("white")  # Set the plot style

        # Reshape labels for compatibility and filter images for the current label
        y = y.reshape(-1, 1, 1)
        boolean_mask = np.any(y == label_to_display, axis=1).reshape(-1)
        arr = X[boolean_mask]

        # Compute the mean and variability for the current label
        avg_img = np.mean(arr, axis=0)
        std_img = np.std(arr, axis=0)
        print(f"==== Label {label_to_display} ====")
        print(f"Image Shape: {avg_img.shape}")

        # Create a figure to display the average and variability images
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=figsize)
        axes[0].set_title(f"Average image for label {label_to_display}")
        axes[0].imshow(avg_img, cmap="gray")
        axes[1].set_title(f"Variability image for label {label_to_display}")
        axes[1].imshow(std_img, cmap="gray")

        # Save or display the figure based on the `save_image` argument
        if save_image:
            plt.savefig(
                f"{file_path}/avg_var_{label_to_display}.png",
                bbox_inches="tight",
                dpi=150,
            )
        else:
            plt.tight_layout()  # Adjust layout for better spacing
            plt.show()
            print("\n")

In [ ]:
plot_mean_variability_per_labels(X=X, y=y, figsize=(12, 5), save_image=True)

In [ ]:
def subset_image_label(X, y, label_to_display):
    """
    Filters the dataset to include only the images that belong to a specific label.
    """

    # Reshape the labels for compatibility and filter for the given label
    y = y.reshape(-1, 1, 1)
    y = y.reshape(-1, 1, 1)
    boolean_mask = np.any(y == label_to_display, axis=1).reshape(-1)
    df = X[boolean_mask]  # Subset the dataset
    return df


def diff_bet_avg_image_labels_data_as_array(
    X, y, label_1, label_2, figsize=(20, 5), save_image=False
):
    """
    Compares the average images between two specified labels.

    - Verifies that both labels exist in the dataset.
    - Calculates the mean image for each label.
    - Computes the difference between the two mean images.
    - Displays or optionally saves a figure with:
        * Average image for label_1
        * Average image for label_2
        * Difference between the two averages
    """
    sns.set_style("white")

    # Validate that both labels are present in the dataset
    if (label_1 not in np.unique(y)) or (label_2 not in np.unique(y)):
        print(f"Either label {label} or label {label_2}, are not in {np.unique(y)} ")
        return

    # Calculate mean from label_1
    images_label = subset_image_label(X, y, label_1)
    label1_avg = np.mean(images_label, axis=0)
    # Calculate mean from label_2
    images_label = subset_image_label(X, y, label_2)
    label2_avg = np.mean(images_label, axis=0)

    # Compute the difference between the two mean images
    difference_mean = label1_avg - label2_avg

    # Create and display a plot with the results
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=figsize)
    axes[0].imshow(label1_avg, cmap="gray")
    axes[0].set_title(f"Average {label_1}")
    axes[1].imshow(label2_avg, cmap="gray")
    axes[1].set_title(f"Average {label_2}")
    axes[2].imshow(difference_mean, cmap="gray")
    axes[2].set_title(f"Difference image: Avg {label_1} & {label_2}")

    # Save the plot to a file if save_image=True, otherwise display it
    if save_image:
        plt.savefig(f"{file_path}/avg_diff.png", bbox_inches="tight", dpi=150)
    else:
        plt.tight_layout()
        plt.show()

In [ ]:
diff_bet_avg_image_labels_data_as_array(
    X=X, y=y, label_1="Healthy", label_2="Infected", figsize=(12, 10), save_image=True
)

### Image Montage

In [ ]:
import itertools
import random

sns.set_style("white")


def image_montage(dir_path, label_to_display, nrows, ncols, figsize=(15, 10)):
    """
    - Verify if the specified label exists in the directory.
    - Ensure the grid size (nrows * ncols) does not exceed the number of available images.
    - Generate a list of indices for placing images in the grid based on the specified rows and columns.
    - Create a figure to display the images, loading and plotting each in the respective grid space.
    """

    labels = os.listdir(dir_path)

    # Check if the specified label exists in the directory
    if label_to_display in labels:

        # Validate that the montage grid can fit the available images
        images_list = os.listdir(dir_path + "/" + label_to_display)
        if nrows * ncols < len(images_list):
            img_idx = random.sample(images_list, nrows * ncols)
        else:
            print(
                f"Reduce the number of rows or columns for the montage. \n"
                f"There are only {len(images_list)} images available. "
                f"You requested a grid for {nrows * ncols} images."
            )
            return

        # Generate grid indices based on the number of rows and columns
        list_rows = range(0, nrows)
        list_cols = range(0, ncols)
        plot_idx = list(itertools.product(list_rows, list_cols))

        # Create a figure and populate it with the selected images
        fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
        for x in range(0, nrows * ncols):
            img = imread(dir_path + "/" + label_to_display + "/" + img_idx[x])
            img_shape = img.shape
            axes[plot_idx[x][0], plot_idx[x][1]].imshow(img)
            axes[plot_idx[x][0], plot_idx[x][1]].set_title(
                f"Width: {img_shape[1]}px, Height: {img_shape[0]}px"
            )
            axes[plot_idx[x][0], plot_idx[x][1]].set_xticks([])
            axes[plot_idx[x][0], plot_idx[x][1]].set_yticks([])
        plt.tight_layout()
        plt.show()

    else:
        # Notify the user if the selected label does not exist
        print(f"The selected label '{label_to_display}' does not exist.")
        print(f"Available labels are: {labels}")

In [ ]:
for label in labels:
    print(label)
    image_montage(
        dir_path=train_path, label_to_display=label, nrows=3, ncols=3, figsize=(10, 15)
    )
    print("\n")

### Healthy VS Infected Comparison

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def compare_healthy_infected(X, y, n_images=5):
    """Healthy と Infected の画像を並べて比較"""

    # ✅ `y` が `0` / `1` の場合、ラベルを変換
    if np.issubdtype(y.dtype, np.number):  # y が数値型なら
        label_map = {0: "Healthy", 1: "Infected"}
        y = np.array([label_map[label] for label in y])

    # ✅ クラスごとのインデックスを取得
    healthy_indices = np.where(y == "Healthy")[0]
    infected_indices = np.where(y == "Infected")[0]

    # ✅ `n_images` をデータ数に合わせる
    n_images = min(n_images, len(healthy_indices), len(infected_indices))

    # ✅ 画像データを `[0,255]` の範囲にスケール変換 (真っ黒回避)
    if X.max() <= 1.0:  # もし `X` の値が `[0,1]` の範囲なら `[0,255]` に変換
        X = (X * 255).astype(np.uint8)

    # ✅ 画像を並べて表示
    for i in range(n_images):
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        # 画像を取得
        healthy_image = X[healthy_indices[i]]
        infected_image = X[infected_indices[i]]

        # 画像を表示
        axes[0].imshow(healthy_image)
        axes[0].set_title("Healthy")
        axes[0].axis("off")

        axes[1].imshow(infected_image)
        axes[1].set_title("Infected")
        axes[1].axis("off")

        plt.tight_layout()
        plt.show()


# ✅ 修正後の関数を実行
compare_healthy_infected(X, y)